In [ ]:
!pip install langchain langchain_google_genai

In [34]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain.schema.output_parser import StrOutputParser
api_key = userdata.get('GOOGLE_API_KEY')

In [35]:
prompt = ChatPromptTemplate.from_template('write some news on {topic} based on your recent updation')
model = ChatGoogleGenerativeAI(google_api_key=api_key, model='gemini-pro')
parser = StrOutputParser()

In [ ]:
chain = prompt | model | parser

In [ ]:
response = chain.invoke({'topic':'google llm gemini ultra'})

In [ ]:
print(response)

**Google Unveils Gemini Ultra, the Next Generation of Its Large Language Model (LLM)**

**Mountain View, California - March 8, 2023** - Google today announced Gemini Ultra, the latest iteration of its groundbreaking large language model (LLM). Gemini Ultra is built on the same foundation as its predecessor, Gemini, but with significant improvements in terms of scale, performance, and capabilities.

**Key Features of Gemini Ultra:**

* **100 Trillion Parameters:** Gemini Ultra boasts an unprecedented 100 trillion parameters, making it one of the largest LLMs in the world. This massive scale allows it to process and generate text with unparalleled accuracy and fluency.
* **Enhanced Generative Capabilities:** Gemini Ultra's generative capabilities have been significantly enhanced, enabling it to produce text that is more coherent, informative, and engaging. It can generate text in a wide range of styles and formats, including articles, stories, poems, and even code.
* **Improved Understan

### **Use of Runnable Map Using Doc Array In Memory Search**

In [ ]:
from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
!pip install docarray

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    texts=["Ashis is Rama Devotee", "Ashis is Krishna Devotee","Shiva and Sati are same","Ashis is Shiva Devotee"],
    embedding=GooglePalmEmbeddings(google_api_key=api_key)
)
retriever = vectorstore.as_retriever()

In [ ]:
!pip install pydantic==1.10.8

In [ ]:
retriever.get_relevant_documents("Who is Ashis?")

[Document(page_content='Ashis is Rama Devotee'),
 Document(page_content='Ashis is Shiva Devotee'),
 Document(page_content='Ashis is Krishna Devotee'),
 Document(page_content='Shiva and Sati are same')]

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=GooglePalmEmbeddings(google_api_key=api_key)
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents(query = "where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({'question':'Who is Ashis?'})

{'context': [Document(page_content='Ashis is Rama Devotee'),
  Document(page_content='Ashis is Shiva Devotee'),
  Document(page_content='Ashis is Krishna Devotee'),
  Document(page_content='Shiva and Sati are same')],
 'question': 'Who is Ashis?'}

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | parser

In [ ]:
chain.invoke({"question": "who is Ashis? For example if A is Devotee of B then who is A ? answer is A is Devotee of B"})

'Ashis is a Devotee of Shiva, Krishna and Rama'

### **Now using ChromaDB**

In [ ]:
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
db = Chroma.from_texts(["Ashis is Rama Devotee", "Ashis is Krishna Devotee","Shiva and Sati are same","Ashis is Shiva Devotee"],
    embedding=GooglePalmEmbeddings(google_api_key=api_key))

In [ ]:
retriever = db.as_retriever()

In [ ]:
retriever.invoke('Who is Ashis')

[Document(page_content='Ashis is Rama Devotee'),
 Document(page_content='Ashis is Shiva Devotee'),
 Document(page_content='Ashis is Krishna Devotee'),
 Document(page_content='Shiva and Sati are same')]

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model |parser

In [ ]:
chain.invoke({'question':'Who is Ashis? example If A is something of B then Who is A ? answer is Something of B'})

### **Practice of Langchain Expression Language:**

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import json
api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
prompt = '''Write about Digital transformation in few lines and out put the response in  below format including curley brackets as shown below
                   {digital_transform : date when transformation happened,
                     key_points : few key points related to transformation}
                    '''
llm = ChatGoogleGenerativeAI(google_api_key=api_key,model = 'gemini-pro')
str_parser = StrOutputParser()

In [ ]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [ ]:
chain = llm | str_parser |json.loads

In [ ]:
# chain = llm | str_parser

In [39]:
chain.invoke(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why did the bear go to the dentist?\n\nTo get his cavity filled.',
 'Why did the frog get a job at a restaurant?\n\nSo he could croak-a-doodle-doo!']

In [ ]:
for t in chain.stream({"topic": "bears"}):
    print(t)

Why did the teddy bear get arrested?

For stuffing a bunny.


In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear go to the doctor? To get his fur coat checked.'

## **Pydantic Syntax**
> Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic.

> They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In [ ]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")
foo_p.name

'Jane'

In [ ]:
class Class(BaseModel):
    students: List[pUser]

In [ ]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

## **Extraction**

In [ ]:
!pip install langchain

In [2]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [4]:
documents = [i.page_content for i in documents]


In [8]:
(documents)

['\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences\n\n\n\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as in

In [ ]:
from langchain.schema.runnable import RunnableLambda

In [ ]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [ ]:
chain = prep | extraction_chain.map() | flatten

In [ ]:
chain.invoke(doc.page_content)

## **Tools in Langchain**

In [2]:
from langchain.agents import tool

In [15]:
@tool
def search(query:str, value: int) -> str:
  """search for weather online"""

  return "42f"

In [12]:
search.name

'search'

In [13]:
search.description

'search(query: str) -> str - search for weather online'

In [16]:
search.args

{'query': {'title': 'Query', 'type': 'string'},
 'value': {'title': 'Value', 'type': 'integer'}}

In [ ]:
!pip install pydantic==1.10.10

In [3]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [4]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [5]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [6]:
search.run("sf")

'42f'

In [7]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [8]:
get_current_temperature.name

'get_current_temperature'

In [9]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [10]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [11]:
from langchain.tools.render import format_tool_to_openai_function
format_tool_to_openai_function(get_current_temperature)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [13]:
get_current_temperature({"latitude": 20.4625, "longitude": 85.8830})

'The current temperature is 23.0°C'

In [ ]:
!pip install wikipedia

In [15]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [16]:
search_wikipedia.name

'search_wikipedia'

In [17]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [20]:
search_wikipedia({"query": "Cuttack"})

'Page: Cuttack\nSummary: Cuttack ( , or officially Kataka in Odia [ˈkɔʈɔkɔ]), is the former capital and the second largest city in the Indian state of Odisha. It is the headquarters of the Cuttack district. The name of the city is an anglicised form of the Odia and Sanskrit name Kataka which literally means The Fort, a reference to the ancient Barabati Fort around which the city initially developed. Cuttack is known as the Millennium City as well as the Silver City due to its history of 1000 years and famous silver filigree works. The Orissa High Court is located there. It is the commercial capital of Odisha which hosts many trading and business houses in and around the city. Cuttack is famous for its Durga Puja which is one of the most important festivals of Odisha. The city is categorised as a Tier-II city as per the ranking system used by Government of India.The old and the most important part of the city is centred on a strip of land between the Kathajodi River and the Mahanadi Riv

In [23]:
search_wikipedia({"query": "Tier 2 city"})

'Page: Classification of Indian cities\nSummary: The classification of Indian cities is a ranking system used by the Government of India to allocate House Rent Allowance (HRA) to public servants employed in cities in India. HRA is also used by the Indian Revenue Service (IRS) to provide income tax exemptions. Cities are classified on the basis of their population, as recommended by the Sixth Central Pay Finance. Under the latest HRA city ranking scheme, most popular media and culture considers only Tier-X cities to be metropolitan in nature. These eight cities are considered India\'s "metros".\n\nPage: Chinese city tier system\nSummary: The Chinese city tier system (Chinese: 中国城市等级制) is an unofficial hierarchical classification of Chinese cities in the People\'s Republic of China (PRC). There are no such official lists in the country, as the Chinese government does not publish or recognize any official definition or a list of cities included in the tier system. However, it is frequentl

In [31]:
functions = [

        search_wikipedia, get_current_temperature
    ]


In [32]:
# model = model.bind(functions = functions)

In [36]:
model.invoke("what is the weather in sf right now")

AIMessage(content='I do not have real-time capabilities and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the current weather conditions in San Francisco.\n\nI recommend checking the National Weather Service website or a reputable weather app for the most up-to-date and accurate weather information.')

In [37]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [ ]:
chain.invoke({"input": "what is the weather in sf right now"})

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

## **Conversational agent**

In [ ]:
tools = [get_current_temperature, search_wikipedia]

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [ ]:
print(result.tool)
print(result.tool_input)

In [ ]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [ ]:
result1.tool

In [ ]:
observation = get_current_temperature(result1.tool_input)

In [ ]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [ ]:
result1.message_log

In [ ]:
format_to_openai_functions([(result1, observation), ])

In [ ]:
result2 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [ ]:
result2

In [ ]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [ ]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [ ]:
run_agent("what is the weather is sf?")

In [ ]:
run_agent("what is langchain?")

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is langchain?"})

In [ ]:
agent_executor.invoke({"input": "my name is bob"})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [ ]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.invoke({"input": "my name is bob"})

## **Create a chatbot**

In [ ]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [ ]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):

    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return

In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard